### experiment what score we get if we predict all 0 or all 1

In [1]:
import torch
import pandas as pd
import numpy as np
import json
import os
import sys
from sklearn.metrics import f1_score


/Users/filipdorm/opt/anaconda3/envs/sinr_icml_og/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sys.path.append('../../')
import datasets
import models
import utils

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# load model
train_params = torch.load('../../pretrained_models/model_an_full_input_enc_sin_cos_hard_cap_num_per_class_1000.pt', map_location='cpu')
model = models.get_model(train_params['params'])
model.load_state_dict(train_params['state_dict'], strict=True)
model = model.to(DEVICE)
model.eval()

ResidualFCNet(
  (class_emb): Linear(in_features=256, out_features=47375, bias=False)
  (feats): Sequential(
    (0): Linear(in_features=4, out_features=256, bias=True)
    (1): ReLU(inplace=True)
    (2): ResLayer(
      (nonlin1): ReLU(inplace=True)
      (nonlin2): ReLU(inplace=True)
      (dropout1): Dropout(p=0.5, inplace=False)
      (w1): Linear(in_features=256, out_features=256, bias=True)
      (w2): Linear(in_features=256, out_features=256, bias=True)
    )
    (3): ResLayer(
      (nonlin1): ReLU(inplace=True)
      (nonlin2): ReLU(inplace=True)
      (dropout1): Dropout(p=0.5, inplace=False)
      (w1): Linear(in_features=256, out_features=256, bias=True)
      (w2): Linear(in_features=256, out_features=256, bias=True)
    )
    (4): ResLayer(
      (nonlin1): ReLU(inplace=True)
      (nonlin2): ReLU(inplace=True)
      (dropout1): Dropout(p=0.5, inplace=False)
      (w1): Linear(in_features=256, out_features=256, bias=True)
      (w2): Linear(in_features=256, out_features=

In [5]:
#load reference from iucn
with open(os.path.join('../../data/eval/iucn/', 'iucn_res_5.json'), 'r') as f:
            data = json.load(f)
species_ids = list((data['taxa_presence'].keys()))

In [6]:
if train_params['params']['input_enc'] in ['env', 'sin_cos_env']:
    raster = datasets.load_env()
else:
    raster = None
enc = utils.CoordEncoder(train_params['params']['input_enc'], raster=raster)

In [7]:
obs_locs = np.array(data['locs'], dtype=np.float32)
obs_locs = torch.from_numpy(obs_locs).to('cpu')
loc_feat = enc.encode(obs_locs)

In [8]:
classes_of_interest = torch.zeros(len(species_ids), dtype=torch.int64)
taxa_ids = torch.zeros(len(species_ids), dtype=torch.int64)
for tt_id, tt in enumerate(species_ids):
    class_of_interest = np.array([train_params['params']['class_to_taxa'].index(int(tt))])
    classes_of_interest[tt_id] = torch.from_numpy(class_of_interest)
    taxa_ids[tt_id] = int(tt)

In [9]:
classes_of_interest

tensor([ 2573,  2744,  2629,  ..., 47345, 47346, 47348])

In [10]:
with torch.no_grad():
    loc_emb = model(loc_feat, return_feats=True)
    wt = model.class_emb.weight[classes_of_interest, :]

In [15]:
loc_emb.shape[0]

2016842

In [14]:
wt[0,:].shape

torch.Size([256])

In [25]:
per_species_f1 = np.zeros((len(taxa_ids),2))
for tt_id, taxa in enumerate(taxa_ids):
    taxa = taxa.item()
    species_locs = data['taxa_presence'].get(str(taxa))

    #test set
    y_test = np.zeros(loc_emb.shape[0], int)
    y_test[species_locs] = 1

    #all zeros
    y_zeros = np.zeros(loc_emb.shape[0], int)

    #all ones
    y_ones = np.ones(loc_emb.shape[0], int)
    
    per_species_f1[tt_id][0] = f1_score(y_test, y_zeros, average='binary')
    per_species_f1[tt_id][1] = f1_score(y_test, y_ones, average='binary')

    if tt_id % 100 == 0:
        print(tt_id)  

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400


In [31]:
print("F1 score all 0s: ", per_species_f1[:,0].mean())
print("F1 score all 1s: ", per_species_f1[:,1].mean())

F1 score all 0s:  0.0
F1 score all 1s:  0.01768321347790135


In [26]:
np.save('./results/baseline_f1_0s_1s.npy', per_species_f1)